Setup Instructions
==============
These instructions need to be run once to teach Jupyter about the software environment we'd like to use for the tutorial. It's important that you've followd all the steps in [HATSatLPCSetup2019](https://twiki.cern.ch/twiki/bin/view/CMS/HATSatLPCSetup2019), particularly the grid certificate instructions!

The following cell just needs to be executed once. If you logout and log back in, the setup will remain.

In [1]:
%%bash
#!/bin/bash

CMSSW_VER="CMSSW_10_6_0"
KERNEL_NAME="hats-sci-py"

set -e
# Get the CMSSW libraries (specifically ROOT)
SCRAM_ARCH=$(ls -d /cvmfs/cms.cern.ch/$(/cvmfs/cms.cern.ch/common/cmsos)*/cms/cmssw/${CMSSW_VER} | tail -n 1 | awk -F / '{ print $4 }')
export SCRAM_ARCH
source /cvmfs/cms.cern.ch/cmsset_default.sh
if [ -d cmssw-env ]; then
    rm -rf cmssw-env
fi
scramv1 project --name cmssw-env CMSSW $CMSSW_VER 

cd cmssw-env
eval `scramv1 runtime -sh`

# Make a wrapper script to load CMSSW python
cat << 'EOF' > bin/python_wrapper.sh
#!/bin/bash
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
source /cvmfs/cms.cern.ch/cmsset_default.sh
cd $DIR
eval `scramv1 runtime -sh`
cd -
exec python "$@"
EOF
chmod +x bin/python_wrapper.sh

# Create the kernel
mkdir -p "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME"
cat << EOF > "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME/kernel.json"
{
 "display_name": "$KERNEL_NAME", 
 "language": "python", 
 "argv": [
  "$PWD/bin/python_wrapper.sh", 
  "-m", 
  "ipykernel_launcher", 
  "-f", 
  "{connection_file}"
 ]
}
EOF

# Report OK
echo "Loaded $CMSSW_VERSION into $KERNEL_NAME!"

Loaded CMSSW_10_6_0 into hats-sci-py!


Results
=======

If successful, you should see something similar to the following:

```
Loaded CMSSW_10_6_0 into hats-sci-py!
WARNING: In non-interactive mode release checks e.g. deprecated releases, production architectures are disabled.
```

The new kernel you just made will then show up in the various Jupyter dropdowns, adding it alongside the stock defaults.